## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

## Лабораторная работа 3

In [ ]:
# !pip install line_profiler

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [2]:
import pandas as pd
recipes = pd.read_csv('recipes_sample.csv', parse_dates=['submitted'])
reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])

In [3]:
%%time
print("A")
def count_rating1():
    summ = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year==2010:
            count+=1
            summ+=row['rating']
    answer1 = summ/count
    return answer1
a = count_rating1()
a

A
CPU times: total: 4.2 s
Wall time: 4.38 s


4.4544402182900615

In [4]:
%%time
print("Б")
def count_rating2():
    summ = 0 
    reviews1 = reviews[reviews['date'].dt.year==2010]
    for index, row in reviews1.iterrows():
        summ+=row['rating']

    answer2 = (summ/reviews1.shape[0])
    return answer2
b = count_rating2()
b

Б
CPU times: total: 422 ms
Wall time: 415 ms


4.4544402182900615

In [5]:
%%time
print("В")
def count_rating3():
    answer3 = reviews[reviews['date'].dt.year==2010].rating.mean()
    return answer3
c = count_rating3()
c

В
CPU times: total: 15.6 ms
Wall time: 11.6 ms


4.4544402182900615

In [6]:
print(a==b==c)

True


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [7]:
!pip install line_profiler

In [8]:
!pip install memory_profiler

In [10]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [11]:
%reload_ext line_profiler

In [12]:
%load_ext memory_profiler

In [13]:
%%writefile demo.py
def count_rating1():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year==2010:
            count+=1
            summ+=row['rating']
    answer1 = summ/count
    return answer1

Writing demo.py


In [15]:
import cProfile, pstats, io


def profile(fnc):
    
    def inner(*args, **kwargs):
        
        pr = cProfile.Profile()
        pr.enable()
        retval = fnc(*args, **kwargs)
        pr.disable()
        s = io.StringIO()
        sortby = 'cumulative'
        ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        ps.print_stats()
        print(s.getvalue())
        return retval

    return inner
@profile
def count_rating1():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    summ = 0
    count = 0
    for index, row in reviews.iterrows():
        if row['date'].year==2010:
            count+=1
            summ+=row['rating']
    answer1 = summ/count
    return answer1

In [16]:
count_rating1()

         16361090 function calls (16107661 primitive calls) in 9.359 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.218    0.218    9.359    9.359 C:\Users\Admin\AppData\Local\Temp\ipykernel_3092\871135636.py:20(count_rating1)
   126697    0.192    0.000    6.990    0.000 C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\frame.py:1279(iterrows)
126702/126699    0.846    0.000    6.634    0.000 C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\series.py:323(__init__)
   126708    0.462    0.000    1.894    0.000 C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\construction.py:470(sanitize_array)
   138790    0.301    0.000    1.609    0.000 C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\series.py:943(__getitem__)
   126699    0.173    0.000    1.288    0.000 C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\internals\managers.py:1731(from_array)
   138790    0.173    0.000    1.

4.4544402182900615

In [17]:
%%time
def count_rating4():
    reviews = pd.read_csv('reviews_sample.csv',index_col = 0,  parse_dates=['date'])
    reviews1 = reviews[reviews['date'].dt.year==2010]
    summ_rating = reviews1.rating.sum()
    return summ_rating/reviews1.shape[0]
count_rating4()

CPU times: total: 562 ms
Wall time: 558 ms


4.4544402182900615

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [19]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [20]:
%%time
get_word_reviews_count(reviews)

CPU times: total: 13.8 s
Wall time: 14.2 s


{'Last': 94,
 'week': 804,
 'whole': 5628,
 'sides': 312,
 'of': 109029,
 'frozen': 2647,
 'salmon': 729,
 'fillet': 60,
 'was': 88781,
 'on': 34583,
 'sale': 149,
 'in': 61539,
 'my': 44144,
 'local': 561,
 'supermarket,': 10,
 'so': 46090,
 'I': 285147,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13965,
 '3,': 48,
 'but': 42513,
 'total': 381,
 'weight': 160,
 'over': 9065,
 '10': 2303,
 'pounds).': 2,
 '': 214145,
 'This': 39448,
 'recipe': 41098,
 'is': 55075,
 'perfect': 4398,
 'for': 121224,
 'fillet,': 14,
 'even': 7878,
 'though': 2314,
 'it': 111175,
 'calls': 520,
 'steaks.': 93,
 'cut': 6688,
 'up': 13585,
 'the': 266050,
 'into': 7031,
 'individual': 314,
 'portions': 156,
 'and': 217849,
 'followed': 4859,
 'instructions': 731,
 'exactly.': 571,
 "I'm": 7145,
 'one': 15086,
 'those': 2287,
 'food': 2413,
 'combining': 74,
 'diets,': 5,
 'left': 4690,
 'out': 23644,
 'white': 3425,
 'wine': 1256,
 'added': 21710,
 'just': 24944,
 'a': 166136,
 'dash': 532,
 'vineg

In [21]:
%%time
def get_word_reviews_count_2(df):
    df['review'] = df['review'].fillna("")
    df['review'] = df['review'].apply(lambda x: ' '.join(set(x.split(' '))))
    return (pd.Series(' '.join(df.review).split()).value_counts())

get_word_reviews_count_2(reviews)

CPU times: total: 2.14 s
Wall time: 2.26 s


I           101778
and          96186
the          92824
a            82412
for          74702
             ...  
cliff            1
QUESTION         1
grease?          1
soy-milk         1
44th             1
Length: 164271, dtype: int64

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [22]:
reviews = reviews[reviews.rating !=0]
mean_rating = reviews.rating.mean()
mean_rating

4.6611588859881055

In [23]:
%%time
print("A")
def mape_A():
    n = reviews.shape[0]
    y = [abs(rating - mean_rating) for rating in reviews.rating]
    percentage_deviation = [(t / mean_rating) * 100 for t in y]
    mape = sum(percentage_deviation) / n
    return mape

mape_A()

A
CPU times: total: 31.2 ms
Wall time: 29.9 ms


11.079150232255099

In [24]:
import numba

In [25]:
%%time
print("B")
@numba.jit(nopython=True)
def mape_B(reviews):
    n = len(reviews)
    mean_rating = sum(reviews) / n
    absolute_errors = []
    for rating in reviews:
        absolute_errors.append(abs(rating - mean_rating))
    percentage_errors = []
    for error in absolute_errors:
        percentage_errors.append(error / mean_rating)
    mape = sum(percentage_errors) * 100 / n
    return mape
mape_B(reviews.rating.values)

B
CPU times: total: 578 ms
Wall time: 945 ms


11.079150232279916

In [26]:
%%time
print("C")
import numpy as np
def mape_C():
    n = reviews.shape[0]
    mean_rating = np.mean(reviews.rating)
    absolute_errors = np.abs(reviews.rating - mean_rating)
    percentage_errors = absolute_errors / mean_rating
    mape = np.sum(percentage_errors) * 100 / n
    return mape
mape_C()

C
CPU times: total: 0 ns
Wall time: 997 µs


11.07915023226724

In [27]:
%%time
import numba

@numba.jit(nopython=True)
def mape_D(ratings_array):
    n = len(ratings_array)
    mean_rating = np.mean(ratings_array)
    absolute_errors = np.abs(ratings_array - mean_rating)
    percentage_errors = absolute_errors / mean_rating
    mape = np.sum(percentage_errors) * 100 / n
    return mape
mape_D(reviews.rating.values)

CPU times: total: 406 ms
Wall time: 423 ms


11.079150232279916